In [1]:
# Sentence encodings
from bert_sent_encoding import bert_sent_encoding


# Import pre-trained Bert for sentence embeds
bse = bert_sent_encoding(model_path='bert-base-cased')

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [2]:
import logging
import os
import queue
import random
import re
from args import get_setup_args
import shutil
import string
#import setup
import torch
import torch.nn.functional as F
import torch.utils.data as data
from collections import Counter
import tqdm
import numpy as np
import ujson as json
#import spacy
import json




In [3]:
def pair_question_contexts():
# Read in train test, dev jsons
# Get each paragraph

# TRAIN
# Split into {question_id -> ("question_string", ["sent1_str", "sent2_str", "sent3_str", ...]))}
# Split into {qid -> (q_embed,[sent1_embed, sent2_embed, sent3_embed, ....])}

#DEV + TEST
# Do same for dev + test

#populate three dictionaries

    datasets = ["dev","test","train"]
    dev_data = train_data = test_data = None
    v_str = "-v2.0.json"

    for ds in datasets:
        filename = ds + v_str
        with open(filename) as f:
            json_file = json.load(f)

                

            # outermost is a keys list of [version, data]
            data = json_file["data"]
            # Data is a list of dicts
            for entry in data:
            
                # Each entry is a dict of keys [title, paragraphs]
#                 print(entry['title']) #Title example is normans
                #Paragraphs is a list of dicts
    #            print(ex_entry['paragraphs'])


                paragraphs = entry['paragraphs']
    #             print(ex_paragraphs[0])
            
                # Want to create new paragraphs list 
                context_to_qa_list = [] # REPLACE paragraphs
                # Each paragraph has a list of questions and associated answers and context
                # each paragraph is a dict with keys [qas, context]  
                #ex_paragraph['context'] is just sent paragraph
    #             print(ex_paragraph['qas'])
                for paragraph in paragraphs:
                    context = paragraph['context']   
                    for qa in paragraph['qas']:
                        # Append to 1:1 dict list here
                        context_to_qa_list.append({'qas':qa,'context':context})
                    
                entry['paragraphs'] = context_to_qa_list
                
                if ds == "dev":
                    dev_data = json_file
#                     print(dev_data['data'][0]['paragraphs'][0])
                elif ds == "test":
                    test_data = json_file
                else:
                    train_data = json_file
            
    return dev_data,train_data,test_data

#             vectors = bse.get_vector(['sent2', 'sent3'], word_vector=False, layer=-1)
#             print(vectors)
#             vectors = bse.get_vector(['sent2', 'sent3'], word_vector=False, layer=-1)  # 4th line 2. get vector list of strings
#            bse.write_txt2vector(input_file, output_file, word_vector=False, layer=-1)   # 5th line 3. get and write vectors of strings


In [5]:
# Maps all dev,train, and test to 1:1 in regards to question and context
# no longer have multiple questions per context, now ea. q-a gets its own context :3
data = pair_question_contexts()


In [27]:
def is_answer_in_sentence(sentence, answers):
    for answer in answers:
        if answer['text'] in sentence:
            return True
    return False
    

In [7]:
import re
from scipy.spatial.distance import cosine as cos_sim
from tqdm import tqdm_notebook

In [56]:
'''
[PLEASE READ: don't need to run this again :)]
Created this because bert was taking a million years on my laptop :)
i'm going to make the the ids for question and sentence pairs as such:
qid + sentence index
'''
from nltk import tokenize
sent_sel_og = {}
dataset_label = ["dev", "train", "test"]
def build_data_og(data):
    for i, dataset in enumerate(data):
        sent_sel_og[dataset_label[i]] = {}
        for entry in dataset['data']:
            paragraphs = entry['paragraphs']
            for paragraph in paragraphs:
                curr_context = paragraph['context']
                curr_qa = paragraph['qas']
                question = curr_qa['question']
                q_id = curr_qa['id']
                answers = curr_qa['answers']
                sentences = tokenize.sent_tokenize(curr_context)
                for s_id, sentence in enumerate(sentences):
                    contains_answer = is_answer_in_sentence(sentence, answers)
                    sent_sel_og[dataset_label[i]][q_id + "_" + str(s_id)] = {"question": question, "sentence": sentence, "contains_answer":contains_answer}
#                 answers = paragraph[]
    return sent_sel_og
        
    

In [57]:
# NO NEED TO RUN AGAIN
huge_data = build_data_og(data)

In [58]:
# [PLEASE READ] no need to run this again!
with open('train_sent_sel_squad.json', 'w') as outfile:
    json.dump(huge_data["train"], outfile)
    
with open('dev_sent_sel_squad.json', 'w') as outfile:
    json.dump(huge_data["dev"], outfile)
    
with open('test_sent_sel_squad.json', 'w') as outfile:
    json.dump(huge_data["test"], outfile)

In [141]:
# FOR NOW COSINE SIM, but NEXT STEP IS REPLACING COS_SIM WITH MODEL PREDICTION
from nltk import tokenize
''' 
train: {
id: {
    sentence: hudfhi
    
    question hflialdn
    
    is_ans_containing = True
    
}
}
'''

sent_sel = {}
def remove_least_relevant_sents(data):
    for dataset in data:
        for entry in dataset['data']:
            paragraphs = entry['paragraphs']
            context_to_qa_list = []
            for paragraph in paragraphs:
                punct = '!.?'
                curr_context = paragraph['context']
                curr_qa = paragraph['qas']
                
                question = curr_qa['question']
                q_embed = bse.get_vector(question, word_vector=False, layer=-1)
                
                # THIS IS IN CASE OF PUNCTUATION INSIDE OF QUOTES(for dialogue) NOT FINISHED 
    #             context_as_sentences = None
    #             if '"' in curr_context or "'" in curr_context:
    #                 lst = []
    #                 prev_index = 0
    #                 has_closed = True
    #                 for i in range(curr_context):
    #                     if curr_context[i] == """""" or curr_context[i] == "''":
    #                         has_closed = not has_closed
    #                     if curr_context[i] in punct:
    #                         context_as_sentences.append(context[prev_index: i])
    #                 context_as_sentences = lst
                
                # Split up context into sentences
#                 context_as_sentences = re.split(r"([!|.|?])+",curr_context)
#                 # The dog. Is smart. Yay!["The dog.", "Is Smart."]
#                 print("curr_context: ", curr_context) 
#                 context_as_sentences, punctuation = context_as_sentences[0::2], context_as_sentences[1::2]
                
                # tokenize context
                sentences = tokenize.sent_tokenize(curr_context)
                print("sentences: ", sentences)
                

                
                
                # Remove empty elems in lst
                # context_as_sentences = list(filter(lambda x: x != "",context_as_sentences))

                # Get context embeddings
                context_embeds = bse.get_vector(sentences, word_vector=False, layer=-1)
                
                #Note that spatial.distance.cosine computes the distance, and not the similarity. 
                #So, you must subtract the value from 1 to get the similarity

                lowest_sent_sim = 1.1 
                sent_index_to_remove = -1
                preds = []
                # Find worst sentence
                for sent_embed in context_embeds:
                    embed_similarity = 1 - cos_sim(q_embed, sent_embed)
                    preds.append(embed_similarity)
#                     if embed_similarity < lowest_sent_sim:
#                         sent_index_to_remove = index
                sent_index_to_remove = np.argmax(np.asarray(preds))
                        
                print(preds)
                # Remove least relevant sentence
                print("CONTEXT IS = " + curr_context)
                print(sent_index_to_remove)
                print("QUESTION is = " + str(curr_qa))
                print("sent removed is = " + str(sentences[sent_index_to_remove]))
                sentences.pop(sent_index_to_remove)
#                 punctuation.pop(sent_index_to_remove)
                
                new_context = ''.join(sentences)
                context_to_qa_list.append({'qas':curr_qa,'context':new_context})
                entry['paragraphs'] = context_to_qa_list
                
    return data  

In [142]:
worst_relevant_sent_removed = remove_least_relevant_sents(data)
print(worst_relevant_sent_removed[0]['data'][0]['paragraphs'])

sentences:  ['The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France.', 'They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia.', 'Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia.', 'The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.']
[0.9762558656009501, 0.8476060774489061, 0.8111153970319515, 0.982571981316717]
CONTEXT IS = The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to No

[0.9796683757285255, 0.8462554591050674, 0.8098116125378627, 0.9869083735711317]
CONTEXT IS = The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
3
QUESTION is = {'question': 'What century did the Normans first gain their separate identity?', 'id': '1be9fa378cbee6d9fa8b6ac78', 'answers': [{'text': '10th cent

[0.9267943229075315, 0.966278771836704, 0.9607347938895401, 0.8974356943870198, 0.946727272262844, 0.853650511248027, 0.7065366360039089]
CONTEXT IS = The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East. The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Catholic orthodoxy into which they assimilated. They adopted the Gallo-Romance language of the Frankish land they settled, their dialect becoming known as Norman, Normaund or Norman French, an important literary language. The Duchy of Normandy, which they formed by treaty with the French crown, was a great fief of medieval France, and under Richard I of Normandy was forged into a cohesive and formidable principality in feudal tenure. The Normans are noted both for their culture, such as their unique Romanesque architecture and musical traditions, and for their significant military accomplishments and innovations

[0.9581890428129378, 0.9841250082649313, 0.9799212379936937, 0.8721325102917293, 0.9713718572186996, 0.8222248546538676, 0.6632091913628498]
CONTEXT IS = The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East. The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Catholic orthodoxy into which they assimilated. They adopted the Gallo-Romance language of the Frankish land they settled, their dialect becoming known as Norman, Normaund or Norman French, an important literary language. The Duchy of Normandy, which they formed by treaty with the French crown, was a great fief of medieval France, and under Richard I of Normandy was forged into a cohesive and formidable principality in feudal tenure. The Normans are noted both for their culture, such as their unique Romanesque architecture and musical traditions, and for their significant military accomplishments and innovati

[0.9261450252754153, 0.9663850955518718, 0.9602295756594359, 0.8990219186117346, 0.9457258119427229, 0.8547869936035931, 0.708065572624959]
CONTEXT IS = The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East. The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Catholic orthodoxy into which they assimilated. They adopted the Gallo-Romance language of the Frankish land they settled, their dialect becoming known as Norman, Normaund or Norman French, an important literary language. The Duchy of Normandy, which they formed by treaty with the French crown, was a great fief of medieval France, and under Richard I of Normandy was forged into a cohesive and formidable principality in feudal tenure. The Normans are noted both for their culture, such as their unique Romanesque architecture and musical traditions, and for their significant military accomplishments and innovatio

...

[0.983807780716948, 0.9627731575617134, 0.9901111522282972, 0.8671807555652483, 0.9288780715912436, 0.9917169774426674, 0.9818801846349529]
CONTEXT IS = The integer factorization problem is the computational problem of determining the prime factorization of a given integer. Phrased as a decision problem, it is the problem of deciding whether the input has a factor less than k. No efficient integer factorization algorithm is known, and this fact forms the basis of several modern cryptographic systems, such as the RSA algorithm. The integer factorization problem is in NP and in co-NP (and even in UP and co-UP). If the problem is NP-complete, the polynomial time hierarchy will collapse to its first level (i.e., NP will equal co-NP). The best known algorithm for integer factorization is the general number field sieve, which takes time O(e(64/9)1/3(n.log 2)1/3(log (n.log 2))2/3) to factor an n-bit integer. However, the best known quantum algorithm for this problem, Shor's algorithm, does ru

[0.9936950234905458, 0.9928497394027845, 0.9927563225118323, 0.9934803078643033, 0.9934389208912164]
CONTEXT IS = Many known complexity classes are suspected to be unequal, but this has not been proved. For instance P ⊆ NP ⊆ PP ⊆ PSPACE, but it is possible that P = PSPACE. If P is not equal to NP, then P is not equal to PSPACE either. Since there are many known complexity classes between P and PSPACE, such as RP, BPP, PP, BQP, MA, PH, etc., it is possible that all these complexity classes collapse to one class. Proving that any of these classes are unequal would be a major breakthrough in complexity theory.
0
QUESTION is = {'question': 'What is the unproven assumption generally ascribed to the value of complexity classes?', 'id': 'dcbbe7cac2aad72ff29bbefc0', 'answers': [{'text': 'suspected to be unequal', 'answer_start': 34}, {'text': 'unequal', 'answer_start': 50}, {'text': 'Many known complexity classes are suspected to be unequal', 'answer_start': 0}], 'is_impossible': False}
sent r

[0.9940156440949074, 0.994837433486299, 0.9903322824036715, 0.993004783687474, 0.9960714045189272]
CONTEXT IS = Many known complexity classes are suspected to be unequal, but this has not been proved. For instance P ⊆ NP ⊆ PP ⊆ PSPACE, but it is possible that P = PSPACE. If P is not equal to NP, then P is not equal to PSPACE either. Since there are many known complexity classes between P and PSPACE, such as RP, BPP, PP, BQP, MA, PH, etc., it is possible that all these complexity classes collapse to one class. Proving that any of these classes are unequal would be a major breakthrough in complexity theory.
4
QUESTION is = {'plausible_answers': [{'text': 'between P and PSPACE', 'answer_start': 269}], 'question': 'Where can complexity classes RPP, BPP, PPP, BQP, MA, and PH be located?', 'id': '8d8670b2f0b17469eb414e5ab', 'answers': [], 'is_impossible': True}
sent removed is = Proving that any of these classes are unequal would be a major breakthrough in complexity theory.
sentences:  ['Ma

[0.9518638510254331, 0.944070083055674, 0.9809483854497782, 0.9294318076006264]
CONTEXT IS = Along the same lines, co-NP is the class containing the complement problems (i.e. problems with the yes/no answers reversed) of NP problems. It is believed that NP is not equal to co-NP; however, it has not yet been proven. It has been shown that if these two complexity classes are not equal then P is not equal to NP.
2
QUESTION is = {'plausible_answers': [{'text': 'co-NP', 'answer_start': 22}], 'question': 'What complexity class do incompatible problems of NP problems exist?', 'id': '40a552de9224483ecf771f4e5', 'answers': [], 'is_impossible': True}
sent removed is = It is believed that NP is not equal to co-NP; however, it has not yet been proven.
sentences:  ['Along the same lines, co-NP is the class containing the complement problems (i.e.', 'problems with the yes/no answers reversed) of NP problems.', 'It is believed that NP is not equal to co-NP; however, it has not yet been proven.', 'It 

IndexError: list index out of range

In [162]:
def adv_pick_most_relevant_sents(data):
    for dataset in data:
        for context_id in dataset:
            print(context_id)
            print(dataset[context_id]["context"])
            paragraph = dataset[context_id]
            curr_context = paragraph['context']
#             curr_qa = paragraph['qas']
                
            question = paragraph['question']
            q_embed = bse.get_vector(question, word_vector=False, layer=-1)
                
            # tokenize context
            sentences = tokenize.sent_tokenize(curr_context)
            print("sentences: ", sentences)

                
            # Remove empty elems in lst
            # context_as_sentences = list(filter(lambda x: x != "",context_as_sentences))

            # Get context embeddings
            context_embeds = bse.get_vector(sentences, word_vector=False, layer=-1)
                
            #Note that spatial.distance.cosine computes the distance, and not the similarity. 
            #So, you must subtract the value from 1 to get the similarity

            lowest_sent_sim = 1.1 
            sent_index_to_remove = -1
            preds = []
            # Find worst sentence
            for sent_embed in context_embeds:
                embed_similarity = 1 - cos_sim(q_embed, sent_embed)
                preds.append(embed_similarity)
            sent_index_to_remove = np.argmax(np.asarray(preds))
                        
            print(preds)
            # Remove least relevant sentence
            print("CONTEXT IS = " + curr_context)
            print(sent_index_to_remove)
            print("QUESTION is = " + str(question))
            print("sent removed is = " + str(sentences[sent_index_to_remove]))
            sentences.pop(sent_index_to_remove)
                
#             new_context = ''.join(sentences)
#             context_to_qa_list.append({'qas':curr_qa,'context':new_context})
#             entry['paragraphs'] = context_to_qa_list
                

In [83]:

# Trying Co-sim on adversarial

adv_dev_eval = adv_train_dev = adv_test_dev = None
with open("adv_dev_eval.json") as f:
    adv_dev_eval = json.load(f)
    
with open("adv_train_eval.json") as f:
    adv_train_eval = json.load(f)

with open("adv_test_eval.json") as f:
    adv_test_eval = json.load(f)
    
# for key in adv_dev_eval.keys():
#     print(key)
data = (adv_dev_eval, adv_train_eval, adv_test_eval)
    

In [84]:
def is_answer_in_sentence_adv(sentence, answers):
    for answer in answers:
        if answer in sentence:
            return True
    return False
    

In [85]:
def build_data_adv(data):
    sent_sel_adv = {}
    dataset_label = ["dev", "train", "test"]
    for i, dataset in enumerate(data):
        sent_sel_adv[dataset_label[i]] = {}
        for context_id in dataset:
            curr_context = dataset[context_id]
            paragraph = curr_context['context']      
            question = curr_context['question']
            sentences = tokenize.sent_tokenize(paragraph)
            q_id = curr_context["uuid"]
            answers = curr_context["answers"]
            for s_id, sentence in enumerate(sentences):
                contains_answer = is_answer_in_sentence_adv(sentence, answers)
                sent_sel_adv[dataset_label[i]][q_id + "_" + str(s_id)] = {"question": question, "sentence": sentence, "contains_answer":contains_answer}
#                 answers = paragraph[]
    return sent_sel_adv

In [87]:
# YOU DON'T NEED TO RUN THIS AGAIN :) 
huge_data_adv = build_data_adv(data)

In [88]:
# [PLEASE READ] no need to run this again!
with open('train_sent_sel_adv.json', 'w') as outfile:
    json.dump(huge_data_adv["train"], outfile)
    
with open('dev_sent_sel_adv.json', 'w') as outfile:
    json.dump(huge_data_adv["dev"], outfile)
    
with open('test_sent_sel_adv.json', 'w') as outfile:
    json.dump(huge_data_adv["test"], outfile)

In [ ]:
best_selected = adv_pick_most_relevant_sents(data)